In [1]:
#Nathan Li - Larger Titanic Survival Model (Final, Simple)
#The only difference between this model and my previous simple model for final submission is that this model uses a larger dataset

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-dataset-from-lake-forest-college/Titanic dataset from Lake Forest College - Sheet1.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
train_data = pd.read_csv("/kaggle/input/titanic-dataset-from-lake-forest-college/Titanic dataset from Lake Forest College - Sheet1.csv")
train_data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0,0,0,24160,211.34,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,1.0,1,2,113781,151.55,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.55,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0,1,2,113781,151.55,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0,1,2,113781,151.55,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score #used instead of MAE because the Titanic problem is a classification type, not regressiosn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
#Dropping the features that will not be used
train_data = train_data.drop(["name", "ticket", "cabin", "boat", "body", "home.dest"], axis = 1)
test_data = test_data.drop(["Name", "Ticket", "Cabin"], axis = 1)

#Capitalize train data columns
train_data = train_data.rename(columns={
    "pclass":"Pclass", 
    "survived":"Survived", 
    "sex":"Sex", 
    "age":"Age", 
    "sibsp":"Sibsp", 
    "parch":"Parch", 
    "fare":"Fare", 
    "embarked":"Embarked"
})

#Simply filling empty Age values with the median age
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())
test_data["Age"] = test_data["Age"].fillna(train_data["Age"].median())

#Filling empty Embarked values with the mode, S
train_data["Embarked"] = train_data["Embarked"].fillna("S")
test_data["Embarked"] = test_data["Embarked"].fillna("S")

#Filling empty Fare value with the median Fare
train_data["Fare"] = train_data["Fare"].fillna(train_data["Fare"].median())
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].median())

In [7]:
le = LabelEncoder()

#Turning the categorical Sex values to numerical
train_data["Sex"] = le.fit_transform(train_data["Sex"])
test_data["Sex"] = le.fit_transform(test_data["Sex"])

#Turning the categorical Embarked values to numerical with a method similar to hot-end encoding
train_data = pd.get_dummies(train_data, columns = ["Embarked"])
test_data = pd.get_dummies(test_data, columns = ["Embarked"])
train_data.head()

,Pclass,Survived,Sex,Age,Sibsp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,1,0,29.0,0,0,211.34,0,0,1
1,1,1,1,1.0,1,2,151.55,0,0,1
2,1,0,0,2.0,1,2,151.55,0,0,1
3,1,0,1,30.0,1,2,151.55,0,0,1
4,1,0,0,25.0,1,2,151.55,0,0,1


In [8]:
#Using train test split
y = train_data["Survived"]
X = train_data.drop("Survived", axis = 1)
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [9]:
#I didn't bother changing the Gradient Boosting Classifier parameters, but I may be able to squeeze out more accuracy by messing around with it later
model = GradientBoostingClassifier(random_state=1)
model.fit(train_X, train_y)
predictions = model.predict(val_X)

print('The accuracy of the Gradient Boosting Classifier is',round(accuracy_score(val_y,predictions)*100,2))

The accuracy of the Gradient Boosting Classifier is 82.06


In [10]:
#Test for submission
X_test = test_data.drop("PassengerId", axis = 1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
